In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import kneighbors_graph
import tensorflow as tf
import zipfile
import os


In [3]:
zip_ref = zipfile.ZipFile("tmp/fold_0.zip", 'r')
zip_ref.extractall("tmp")
zip_ref.close()

In [5]:
zip_ref = zipfile.ZipFile("tmp/fold_1.zip", 'r')
zip_ref.extractall("tmp")
zip_ref.close()

In [16]:
doente_dir = os.path.join('tmp/fold_0/all')
saudavel_dir = os.path.join('tmp/fold_1/hem')

In [17]:
print('total de imagens doente:', len(os.listdir(doente_dir)))
print('total de imagens saudavel:', len(os.listdir(saudavel_dir)))

total de imagens doente: 2397
total de imagens saudavel: 1163


In [ ]:
# Visualizar Array de imagens

doente_files = os.listdir(doente_dir)
print(doente_files[:10])

saudavel_files = os.listdir(saudavel_dir)
print(saudavel_files[:10])

In [19]:
#Criando Dataset
dataset_dir = os.path.join(os.getcwd(), 'tmp')
dataset_train_dir = os.path.join(dataset_dir, 'fold_0') 
dataset_train_doente_len = len(os.listdir(os.path.join(dataset_train_dir, 'all'))) 
dataset_train_saudavel_len = len(os.listdir(os.path.join(dataset_train_dir, 'hem')))

dataset_validation_dir = os.path.join(dataset_dir, 'fold_1')
dataset_validation_doente_len = len(os.listdir(os.path.join(dataset_validation_dir, 'all'))) 
dataset_validation_saudavel_len = len(os.listdir(os.path.join(dataset_validation_dir, 'hem')))

print('Train Doente: %s' % dataset_train_doente_len) 
print('Train Saudavel: %s' % dataset_train_saudavel_len) 
print('Validation Doente: %s' % dataset_validation_doente_len) 
print('Validation Saudavel: %s' % dataset_validation_saudavel_len)

Train Doente: 2397
Train Saudavel: 1130
Validation Doente: 2418
Validation Saudavel: 1163


In [20]:
#Redimencionamento de Imagem

image_width = 160
image_height = 160
image_color_channel = 3
image_color_channel_size = 255
image_size = (image_width, image_height)
image_shape = image_size + (image_color_channel,)

batch_size = 32
epochs = 20
learning_rate = 0.0001

class_names = ['doente', 'saudavel']

In [21]:
#Dataset de Treino

dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_train_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

Found 3527 files belonging to 2 classes.


In [22]:
#Dataset de Validação

dataset_validation = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_validation_dir,
    image_size = (image_width, image_height),
    batch_size = batch_size,
    shuffle = True
)

Found 3581 files belonging to 2 classes.


In [23]:
#Aplicando Cardinalidade

dataset_validation_cardinality = tf.data.experimental.cardinality(dataset_validation)
dataset_validation_batches = dataset_validation_cardinality // 5

dataset_test = dataset_validation.take(dataset_validation_batches)
dataset_validation = dataset_validation.skip(dataset_validation_batches)

print('Validation Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_validation))
print('Test Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_test))

Validation Dataset Cardinality: 90
Test Dataset Cardinality: 22


In [24]:
#Aplicando AutoTune

autotune = tf.data.AUTOTUNE

dataset_train = dataset_train.prefetch(buffer_size = autotune)
dataset_validation = dataset_validation.prefetch(buffer_size = autotune)
dataset_test = dataset_validation.prefetch(buffer_size = autotune)

In [25]:
#Plotar Dataset

def plot_dataset(dataset):

    plt.gcf().clear()
    plt.figure(figsize = (15, 15))

    for features, labels in dataset.take(1):

        for i in range(9):

            plt.subplot(3, 3, i + 1)
            plt.axis('off')

            plt.imshow(features[i].numpy().astype('uint8'))
            plt.title(class_names[labels[i]])

In [ ]:
#Plotar Dataset de Treino

plot_dataset(dataset_train)

In [ ]:
#Plotar Dataset de Validação

plot_dataset(dataset_validation)

In [ ]:
#Plotar dataset de Teste

plot_dataset(dataset_test)

In [30]:
#Aplicando Augmentation

data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

def plot_dataset_data_augmentation(dataset):

    plt.gcf().clear()
    plt.figure(figsize = (15, 15))

    for features, _ in dataset.take(1):

        feature = features[0]

        for i in range(9):

            feature_data_augmentation = data_augmentation(tf.expand_dims(feature, 0))

            plt.subplot(3, 3, i + 1)
            plt.axis('off')

            plt.imshow(feature_data_augmentation[0] / image_color_channel_size)

In [ ]:
#Plotar Dataset Augmentation

plot_dataset_data_augmentation(dataset_train)

In [32]:
#Aplicando LAYER RESCALING

rescaling = tf.keras.layers.experimental.preprocessing.Rescaling(1. / (image_color_channel_size / 2.), offset = -1, input_shape = image_shape)

In [ ]:
#Aplicando TRANSFER LEARNING 

model_transfer_learning = tf.keras.applications.MobileNetV2(input_shape = image_shape, include_top = False, weights = 'imagenet')
model_transfer_learning.trainable = False

model_transfer_learning.summary()

In [34]:
#Aplicando EARLY STOPPING

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

In [ ]:
#Criando o Modelo

model = tf.keras.models.Sequential([
    rescaling,
    data_augmentation,
    model_transfer_learning,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy']
)

model.summary()

In [ ]:
# Aplicando Fit History

history = model.fit(
    dataset_train,
    validation_data = dataset_validation,
    epochs = epochs,
    callbacks = [
        early_stopping
    ]
)

In [38]:
# Plotar Modelo

def plot_model():

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.gcf().clear()
    plt.figure(figsize = (15, 8))

    plt.subplot(1, 2, 1)
    plt.title('Training and Validation Accuracy')
    plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
    plt.plot(epochs_range, val_accuracy, label = 'Validation Accuracy')
    plt.legend(loc = 'lower right')

    plt.subplot(1, 2, 2)
    plt.title('Training and Validation Loss')
    plt.plot(epochs_range, loss, label = 'Training Loss')
    plt.plot(epochs_range, val_loss, label = 'Validation Loss')
    plt.legend(loc = 'lower right')

    plt.show()
    
    plot_model()

In [ ]:
# Avaliando Modelo

dataset_test_loss, dataset_test_accuracy = model.evaluate(dataset_test)

print('Dataset Test Loss:     %s' % dataset_test_loss)
print('Dataset Test Accuracy: %s' % dataset_test_accuracy)

In [ ]:
# Plotar Dataset de Predições

def plot_dataset_predictions(dataset):

    features, labels = dataset_test.as_numpy_iterator().next()

    predictions = model.predict_on_batch(features).flatten()
    predictions = tf.where(predictions < 0.5, 0, 1)

    print('Labels:      %s' % labels)
    print('Predictions: %s' % predictions.numpy())

    plt.gcf().clear()
    plt.figure(figsize = (15, 15))

    for i in range(9):

        plt.subplot(3, 3, i + 1)
        plt.axis('off')

        plt.imshow(features[i].astype('uint8'))
        plt.title(class_names[predictions[i]])
        
plot_dataset_predictions(dataset_test)

In [ ]:
#Salvando

model.save('model')

In [7]:
# Função de Predição

def predict(image_file,size_image):

    #image = tf.keras.preprocessing.image.load_img(image_file, target_size = image_size)
    image = tf.keras.preprocessing.image.load_img(image_file, target_size = size_image)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.expand_dims(image, 0)

    prediction = model.predict(image)[0][0]

    print('Prediction: {0} | {1}'.format(prediction, ('Doente' if prediction < 0.5 else 'Saudavel')))

In [5]:
#Subindo o Modelo
model = tf.keras.models.load_model('model')

In [ ]:
# Teste de Predição de Imagem

image_width = 160
image_height = 160
size_image = (image_width, image_height)


predict("tmp/fold_1/hem/UID_H10_107_2_hem.bmp",size_image)
